# Data Mining Project - Bank

## Project Developed by:
- *Duarte Sardão*
- *Gabriel Ferreira*
- *Miguel Lopes*
- *Sérgio Estêvão*

## Table of Contents
1. Business Understanding

### Business Understanding



In [1]:
import pandas as pd
from datetime import datetime

In [2]:
loans = pd.read_csv("./data/loan_dev.csv", sep=";")
transactions = pd.read_csv("./data/trans_dev.csv", sep=";")
accounts = pd.read_csv("./data/account.csv", sep=";")
cards = pd.read_csv("./data/card_dev.csv", sep=";")
clients = pd.read_csv("./data/client.csv", sep=";")
dispositions = pd.read_csv("./data/disp.csv", sep=";")
districts = pd.read_csv("./data/district.csv", sep=";")

print("transactions: ", len(transactions))
print("dispositions: ", len(dispositions))
print("clients: ", len(clients))
print("cards: ", len(cards))

#Clean up information not associated with loans

for id in loans['account_id'].unique():
    transactions = transactions[transactions.account_id != id]
    dispositions = dispositions[dispositions.account_id != id]

for client_id in dispositions['client_id'].unique():
    clients = clients[clients.client_id != client_id]

for disp_id in dispositions['disp_id'].unique():
    cards = cards[cards.disp_id != disp_id]

print("==== After clean-up ====")
print("transactions: ", len(transactions))
print("dispositions: ", len(dispositions))
print("clients: ", len(clients))
print("cards: ", len(cards))


C:\Users\Duarte\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


transactions:  396685
dispositions:  5369
clients:  5369
cards:  177
==== After clean-up ====
transactions:  372191
dispositions:  4966
clients:  403
cards:  11


In [3]:
from datetime import datetime
#transform birth number into birthdate and sex
birth_number = clients["birth_number"]
clients["sex"] = birth_number.map(lambda nr: "Male" if nr % 10000 < 5000 else "Female").astype('category')
clients["birthdate"] = birth_number.map(lambda nr: str(nr) if nr % 10000 < 5000 else str(nr-5000))
clients["birthdate"] = pd.to_datetime("19"+clients["birthdate"], format='%Y%m%d')
clients.drop(columns="birth_number", inplace=True)
clients.head()


,client_id,district_id,sex,birthdate
1,2,1,Male,1945-02-04
2,3,1,Female,1940-10-09
24,25,21,Female,1939-04-23
73,78,16,Female,1944-06-13
148,158,40,Female,1945-07-03


In [4]:
#other date treatment (yymmdd string to datetime format)
cards["issued"] = cards["issued"].map(lambda nr: str(nr))
cards["issued"] = pd.to_datetime("19"+cards["issued"], format='%Y%m%d')

transactions["date"] = transactions["date"].map(lambda nr: str(nr))
transactions["date"] = pd.to_datetime("19"+transactions["date"], format='%Y%m%d')

loans["date"] = loans["date"].map(lambda nr: str(nr))
loans["date"] = pd.to_datetime("19"+loans["date"], format='%Y%m%d')

accounts["date"] = accounts["date"].map(lambda nr: str(nr))
accounts["date"] = pd.to_datetime("19"+accounts["date"], format='%Y%m%d')


In [5]:
#replace unknown val in 95 with 96 val
districts["no. of commited crimes '95 "] = districts.apply(lambda row:  int(row["no. of commited crimes '95 "]) if row["no. of commited crimes '95 "] != "?" else row["no. of commited crimes '96 "] , axis=1)
districts['crimerate95'] = districts.apply(lambda row: row["no. of commited crimes '95 "] / row["no. of inhabitants"], axis=1)
districts['crimerate96'] = districts.apply(lambda row: row["no. of commited crimes '96 "] / row["no. of inhabitants"], axis=1)
districts.drop(columns="no. of commited crimes '95 ", inplace=True)
districts.drop(columns="no. of commited crimes '96 ", inplace=True)
districts.head()

#adicionar ao cliente as colunas: "rendimentos no ultimo mês/ 6 meses/ano"

,code,name,region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,crimerate95,crimerate96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,0.071104,0.082250
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,0.024290,0.030084
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,0.037537,0.037391
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,0.034985,0.039308
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,0.027359,0.031794
